# Tγ2: Ordered Branch Extended (λ → 0.90)

**Purpose**: Determine upper bound - at what λ does ordered state collapse?

**Protocol**:
- Load ordered checkpoints
- Sweep λ: 0.30 → 0.90 (extended range)
- 3 epochs per λ

**Hypothesis**: Ordered state remains stable even at very high λ

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os, glob, json, time
from datetime import datetime

EXP_NAME = 'exp_Tg2_ordered_extended'
NOTEBOOK_ID = 'Tg2'
BASE_DIR = '/content/drive/MyDrive/dual-gradient-learning/Paper-A'

ORDERED_CKPT_DIR = sorted(glob.glob(f'{BASE_DIR}/exp_Ta_prep_ordered_*'))[-1] + '/checkpoints'
print(f'Ordered checkpoints: {ORDERED_CKPT_DIR}')

existing = glob.glob(f'{BASE_DIR}/{EXP_NAME}_*')
if existing:
    SAVE_DIR = sorted(existing)[-1]
    print(f'🔄 Resuming: {SAVE_DIR}')
else:
    TIMESTAMP = datetime.now().strftime('%Y%m%d_%H%M%S')
    SAVE_DIR = f'{BASE_DIR}/{EXP_NAME}_{TIMESTAMP}'
    os.makedirs(SAVE_DIR, exist_ok=True)
    print(f'🆕 New: {SAVE_DIR}')

os.makedirs(f'{SAVE_DIR}/figures', exist_ok=True)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils import parameters_to_vector
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet18
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')

In [ ]:
BATCH_SIZE = 256
NUM_WORKERS = 4
LR = 0.1
K = 16

NOISE_RATE = 0.4
LAMBDA_START = 0.30
LAMBDA_END = 0.90  # Extended up to 0.90
LAMBDA_STEP = 0.02
EPOCHS_PER_LAMBDA = 3

LAMBDA_GRID = np.arange(LAMBDA_START, LAMBDA_END + LAMBDA_STEP/2, LAMBDA_STEP)
LAMBDA_GRID = np.round(LAMBDA_GRID, 2)

N_SEEDS = 5

print(f'λ sweep: {LAMBDA_START} → {LAMBDA_END}')
print(f'λ points: {len(LAMBDA_GRID)}')
print(f'Total epochs per run: {len(LAMBDA_GRID) * EPOCHS_PER_LAMBDA}')

In [ ]:
def get_resnet18():
    model = resnet18(weights=None, num_classes=10)
    model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
    model.maxpool = nn.Identity()
    return model

class IndexedDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
    def __getitem__(self, idx):
        img, label = self.dataset[idx]
        return img, label, idx
    def __len__(self):
        return len(self.dataset)

def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def inject_label_noise(labels, noise_rate, seed):
    np.random.seed(seed)
    noisy = labels.copy()
    n_noisy = int(noise_rate * len(labels))
    idx = np.random.choice(len(labels), n_noisy, replace=False)
    for i in idx:
        noisy[i] = np.random.choice([l for l in range(10) if l != labels[i]])
    return noisy

def load_cifar10():
    tr = transforms.Compose([transforms.RandomCrop(32, padding=4), transforms.RandomHorizontalFlip(),
                             transforms.ToTensor(), transforms.Normalize((0.4914,0.4822,0.4465),(0.2023,0.1994,0.2010))])
    te = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914,0.4822,0.4465),(0.2023,0.1994,0.2010))])
    return torchvision.datasets.CIFAR10('./data', True, tr, download=True), torchvision.datasets.CIFAR10('./data', False, te, download=True)

def evaluate(model, loader):
    model.eval()
    correct = total = 0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            correct += (model(x).argmax(1) == y).sum().item()
            total += y.size(0)
    return correct / total

In [ ]:
def train_one_epoch(model, train_loader, opt, clean_t, noisy_t, lam, cached_gv_ref):
    crit = nn.CrossEntropyLoss()
    model.train()
    step = cached_gv_ref['step']
    cached_gv = cached_gv_ref['gv']
    
    for x, _, idx in train_loader:
        x, idx = x.to(device), idx.to(device)
        bn, bc = noisy_t[idx], clean_t[idx]
        
        opt.zero_grad()
        loss_s = crit(model(x), bn)
        loss_s.backward(retain_graph=True)
        gs = parameters_to_vector([p.grad for p in model.parameters()]).clone()
        
        if step % K == 0 or cached_gv is None:
            opt.zero_grad()
            loss_v = crit(model(x), bc)
            loss_v.backward()
            cached_gv = parameters_to_vector([p.grad for p in model.parameters()]).clone()
        
        gs_n = gs / (gs.norm() + 1e-12)
        gv_n = cached_gv / (cached_gv.norm() + 1e-12)
        
        g_mix = (1 - lam) * gs_n + lam * gv_n
        opt.zero_grad()
        i = 0
        for p in model.parameters():
            n = p.numel()
            p.grad = g_mix[i:i+n].view(p.shape).clone()
            i += n
        opt.step()
        step += 1
    
    cached_gv_ref['step'] = step
    cached_gv_ref['gv'] = cached_gv

In [ ]:
def run_extended_sweep(ckpt_path, train_loader, test_loader, clean_labels, noisy_labels):
    ckpt = torch.load(ckpt_path, map_location=device)
    seed = ckpt['seed']
    init_error = ckpt['final_error']
    
    print(f'    Loaded: seed={seed}, init_error={init_error:.4f}')
    
    clean_t = torch.tensor(clean_labels, device=device)
    noisy_t = torch.tensor(noisy_labels, device=device)
    
    set_seed(seed + 2000)
    model = get_resnet18().to(device)
    model.load_state_dict(ckpt['model_state_dict'])
    
    opt = optim.SGD(model.parameters(), lr=LR * 0.01, momentum=0.9, weight_decay=5e-4)
    cached_gv_ref = {'step': 0, 'gv': None}
    trajectory = []
    collapse_lambda = None
    
    print(f'    Sweeping λ: {LAMBDA_START} → {LAMBDA_END}...')
    
    for lam in LAMBDA_GRID:
        for _ in range(EPOCHS_PER_LAMBDA):
            train_one_epoch(model, train_loader, opt, clean_t, noisy_t, lam, cached_gv_ref)
        
        err = 1 - evaluate(model, test_loader)
        trajectory.append({'lambda': float(lam), 'error': err})
        
        # Check for collapse
        status = ''
        if err >= 0.40 and collapse_lambda is None:
            collapse_lambda = lam
            status = ' 💀 COLLAPSE!'
        print(f'      λ={lam:.2f}: err={err:.4f}{status}')
    
    return {
        'seed': seed,
        'init_error': init_error,
        'final_error': trajectory[-1]['error'],
        'collapse_lambda': collapse_lambda,
        'trajectory': trajectory,
        'checkpoint_source': os.path.basename(ckpt_path)
    }

In [ ]:
trainset, testset = load_cifar10()
clean_labels = np.array(trainset.targets)
train_loader = DataLoader(IndexedDataset(trainset), BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True)
test_loader = DataLoader(testset, BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)

ordered_ckpts = sorted(glob.glob(f'{ORDERED_CKPT_DIR}/ordered_seed*.pth'))[:N_SEEDS]
print(f'Using {len(ordered_ckpts)} ordered checkpoints')

m = get_resnet18().to(device)
for _ in range(10): _ = m(torch.randn(BATCH_SIZE,3,32,32,device=device))
del m; torch.cuda.empty_cache()
print('Ready')

In [ ]:
results = []
ckpt_file = f'{SAVE_DIR}/{NOTEBOOK_ID}_checkpoint.json'
done_ckpts = set()

if os.path.exists(ckpt_file):
    results = json.load(open(ckpt_file))
    done_ckpts = {r['checkpoint_source'] for r in results}
    print(f'Loaded: {len(done_ckpts)} done')

for i, ckpt_path in enumerate(ordered_ckpts):
    ckpt_basename = os.path.basename(ckpt_path)
    if ckpt_basename in done_ckpts:
        continue
    
    print(f'\n{"="*60}')
    print(f'[{i+1}/{len(ordered_ckpts)}] {ckpt_basename} | {NOTEBOOK_ID}')
    print(f'{"="*60}')
    
    ckpt_temp = torch.load(ckpt_path, map_location='cpu')
    seed = ckpt_temp['seed']
    noisy_labels = inject_label_noise(clean_labels, NOISE_RATE, seed)
    
    t0 = time.time()
    result = run_extended_sweep(ckpt_path, train_loader, test_loader, clean_labels, noisy_labels)
    result['time'] = time.time() - t0
    result['experiment_id'] = f'{NOTEBOOK_ID}-{i+1:03d}'
    
    results.append(result)
    done_ckpts.add(ckpt_basename)
    json.dump(results, open(ckpt_file, 'w'), indent=2, default=str)
    
    print(f'\n  Init: {result["init_error"]:.2%} → Final @ λ=0.90: {result["final_error"]:.2%} | {result["time"]/60:.1f}min')
    if result['collapse_lambda']:
        print(f'  💀 Collapsed at λ={result["collapse_lambda"]}')
    else:
        print(f'  ✅ No collapse up to λ=0.90')
    
    torch.cuda.empty_cache()

print('\n' + '='*60)
print(f'{NOTEBOOK_ID} COMPLETE')
print('='*60)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

json.dump(results, open(f'{SAVE_DIR}/{NOTEBOOK_ID}_results.json', 'w'), indent=2, default=str)

all_data = []
for r in results:
    for t in r['trajectory']:
        all_data.append({'seed': r['seed'], 'lambda': t['lambda'], 'error': t['error']})
df = pd.DataFrame(all_data)
df.to_csv(f'{SAVE_DIR}/{NOTEBOOK_ID}_results.csv', index=False)

# Summary
print('='*60)
print(f'{NOTEBOOK_ID} SUMMARY: Collapse Upper Bound')
print('='*60)

collapse_lambdas = [r['collapse_lambda'] for r in results if r['collapse_lambda'] is not None]
no_collapse = [r for r in results if r['collapse_lambda'] is None]

print(f'\n📊 Collapse Statistics:')
print(f'   Collapsed: {len(collapse_lambdas)}/{len(results)}')
print(f'   No collapse: {len(no_collapse)}/{len(results)}')

if collapse_lambdas:
    print(f'\n   Collapse λ values: {collapse_lambdas}')
    print(f'   Mean collapse λ: {np.mean(collapse_lambdas):.2f}')
    print(f'   Min collapse λ: {np.min(collapse_lambdas):.2f}')

final_errors = [r['final_error'] for r in results]
print(f'\n📊 Final error @ λ=0.90:')
print(f'   Mean: {np.mean(final_errors):.2%}')
print(f'   Std:  {np.std(final_errors):.2%}')

In [ ]:
# Visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Individual trajectories
ax = axes[0]
for r in results:
    lams = [t['lambda'] for t in r['trajectory']]
    errs = [t['error'] for t in r['trajectory']]
    style = 'o-' if r['collapse_lambda'] is None else 'x--'
    ax.plot(lams, errs, style, alpha=0.7, linewidth=2, markersize=4, 
            label=f"seed {r['seed']}" + (f" (collapse@{r['collapse_lambda']})" if r['collapse_lambda'] else ''))

ax.axhline(0.40, color='red', linestyle='--', alpha=0.5, label='Collapse threshold')
ax.axhline(0.20, color='green', linestyle='--', alpha=0.5, label='Ordered threshold')
ax.axvline(0.70, color='gray', linestyle=':', alpha=0.5, label='Original range')
ax.set_xlabel('λ', fontsize=12)
ax.set_ylabel('Test Error', fontsize=12)
ax.set_title(f'Ordered Branch Extended (λ → 0.90)', fontsize=12)
ax.legend(fontsize=8)
ax.grid(True, alpha=0.3)
ax.set_xlim(0.28, 0.92)

# Mean ± std
ax = axes[1]
df_mean = df.groupby('lambda')['error'].agg(['mean', 'std']).reset_index()
ax.fill_between(df_mean['lambda'], df_mean['mean'] - df_mean['std'], 
                df_mean['mean'] + df_mean['std'], alpha=0.3, color='blue')
ax.plot(df_mean['lambda'], df_mean['mean'], 'b-o', linewidth=2, markersize=5)

ax.axhline(0.40, color='red', linestyle='--', alpha=0.5, label='Collapse')
ax.axhline(0.20, color='green', linestyle='--', alpha=0.5, label='Ordered')
ax.axvline(0.70, color='gray', linestyle=':', alpha=0.5, label='Original range')

ax.set_xlabel('λ', fontsize=12)
ax.set_ylabel('Test Error', fontsize=12)
ax.set_title('Mean ± Std', fontsize=12)
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3)
ax.set_xlim(0.28, 0.92)

plt.tight_layout()
plt.savefig(f'{SAVE_DIR}/figures/{NOTEBOOK_ID}_extended_sweep.png', dpi=150)
plt.show()